<a href="https://colab.research.google.com/github/pritika-vig/modeling-adrs/blob/main/feature_engineering/generate_input_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install --quiet google-cloud-bigquery pandas

In [ ]:
# Authenticate with Google Cloud
from google.colab import auth
auth.authenticate_user()

In [ ]:

from google.cloud import bigquery
import pandas as pd
import numpy as np


In [ ]:
# Set up BigQuery client
project_id = "mit-mlhc-class-450518"
client = bigquery.Client(project=project_id)

In [ ]:
# SQL query to get ICU stays that are at least 4 days long
query = """
WITH long_stays AS (
  SELECT icustay_id, subject_id, hadm_id, intime, outtime
  FROM `physionet-data.mimiciii_clinical.icustays`
  WHERE DATETIME_DIFF(outtime, intime, DAY) >= 4
),
input_events_mv AS (
  SELECT icustay_id, itemid, starttime AS event_time
  FROM `physionet-data.mimiciii_clinical.inputevents_mv`
  WHERE icustay_id IS NOT NULL
),
input_events_cv AS (
  SELECT icustay_id, itemid, charttime AS event_time
  FROM `physionet-data.mimiciii_clinical.inputevents_cv`
  WHERE icustay_id IS NOT NULL
),
output_events AS (
  SELECT icustay_id, itemid, charttime AS event_time
  FROM `physionet-data.mimiciii_clinical.outputevents`
  WHERE icustay_id IS NOT NULL
),
all_events AS (
  SELECT * FROM input_events_mv
  UNION ALL
  SELECT * FROM input_events_cv
  UNION ALL
  SELECT * FROM output_events
),
events_3day AS (
  SELECT e.icustay_id, e.itemid
  FROM all_events e
  JOIN long_stays s ON e.icustay_id = s.icustay_id
  WHERE e.event_time BETWEEN s.intime AND TIMESTAMP_ADD(s.intime, INTERVAL 3 DAY)
)
SELECT icustay_id, itemid
FROM events_3day
"""

# Run the query and load into DataFrame
job = client.query(query)
df = job.to_dataframe()

In [ ]:
# MIMIC-III ICU Input/Output Events One-Hot Encoding for First 3 Days


df_grouped = pd.crosstab(df['icustay_id'], df['itemid'].astype(str)).astype(pd.SparseDtype("int", fill_value=0))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Replace with your actual path
label_path = '/content/drive/My Drive/physionet/adr_icustay.csv'
df_labels = pd.read_csv(label_path)

# Ensure icustay_id is the correct type
df_labels['icustay_id'] = df_labels['icustay_id'].astype(df_grouped.index.dtype)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_combined = df_grouped.join(df_labels.set_index('icustay_id'), how='inner')

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

# Define features (X) and labels (y)
X = df_combined.drop(columns=['ADR'])
y = df_combined['ADR']

# Use linear SVM for feature selection
svc = LinearSVC(C=0.01, penalty="l1", dual=False, max_iter=5000).fit(X, y)
model = SelectFromModel(svc, prefit=True)

# Reduce feature set
X_selected = model.transform(X)

# Optional: get selected feature names
selected_features = X.columns[model.get_support()]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [ ]:

selected_mask = model.get_support()
selected_features = X.columns[selected_mask]
selected_coefs = svc.coef_[0][selected_mask]

# Map itemids to human-readable labels
selected_features_readable = [itemid_to_label.get(fid, fid) for fid in selected_features]

# Create DataFrame with readable names
coef_df = pd.DataFrame({
    'feature': selected_features_readable,
    'coefficient': selected_coefs
})

# Sort by coefficient values
top_positive = coef_df.sort_values(by='coefficient', ascending=False).head(10)
top_negative = coef_df.sort_values(by='coefficient', ascending=True).head(10)

# Print results
print("🔺 Top 10 features predicting ADR=1:")
print(top_positive)

print("\n🔻 Top 10 features predicting ADR=0:")
print(top_negative)

🔺 Top 10 features predicting ADR=1:
                  feature  coefficient
27              Acyclovir     0.030691
161        NA Acetate/H2O     0.017703
69        Glucerna (Full)     0.017520
4       Calcium Gluconate     0.015021
158          Protonix gtt     0.014738
160  cerebral drain right     0.013906
34          TPN w/ Lipids     0.013571
66       Acetaminophen-IV     0.012166
35     TPN without Lipids     0.007748
56             Pigtail #2     0.007468

🔻 Top 10 features predicting ADR=0:
                      feature  coefficient
28                 Ampicillin    -0.005459
42                   GT Flush    -0.004104
50                  JP Medial    -0.004016
60  Magnesium Sulfate (Bolus)    -0.003356
48              Chest Tube #2    -0.002810
8                    Dopamine    -0.002795
2                  Amiodarone    -0.002213
64                   Dilantin    -0.002064
88           Free Water Bolus    -0.002053
58                TF Residual    -0.001695


In [ ]:
query_ditems = """
SELECT itemid, label
FROM `physionet-data.mimiciii_clinical.d_items`
"""

df_ditems = client.query(query_ditems).to_dataframe()
df_ditems['itemid'] = df_ditems['itemid'].astype(str)  # Match dtype

In [ ]:
itemid_to_label = dict(zip(df_ditems['itemid'], df_ditems['label']))

In [ ]:
X_selected_df = pd.DataFrame.sparse.from_spmatrix(
    X_selected,
    index=df_combined.index,
    columns=selected_features
)

# Now rename columns using d_items mapping
X_renamed = X_selected_df.rename(columns=itemid_to_label)

In [ ]:
df_out = X_renamed.reset_index()
df_out.to_csv('/content/drive/My Drive/physionet/input_output_features.csv', index=False)
